In [1]:
import pandas as pd

df = pd.read_csv('merged-dataset.csv', parse_dates=['start_date', 'end_date', 'inv_item_date'])

/var/folders/pn/0bn7dksx05d8k5q8rpx3vxm80000gn/T/ipykernel_26170/1993625587.py:3: DtypeWarning: Columns (12,14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('merged-dataset.csv', parse_dates=['start_date', 'end_date', 'inv_item_date'])
/var/folders/pn/0bn7dksx05d8k5q8rpx3vxm80000gn/T/ipykernel_26170/1993625587.py:3: UserWarning: Parsing dates in %d.%m.%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df = pd.read_csv('merged-dataset.csv', parse_dates=['start_date', 'end_date', 'inv_item_date'])
/var/folders/pn/0bn7dksx05d8k5q8rpx3vxm80000gn/T/ipykernel_26170/1993625587.py:3: UserWarning: Parsing dates in %d.%m.%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df = pd.read_csv('merged-dataset.csv', parse_dates=['start_date', 'end_date', 'inv_item_date'])


In [2]:
df = df.drop(columns=['Unnamed: 0'])

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 228636 entries, 0 to 228635
Data columns (total 40 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   season               228636 non-null  int64         
 1   field                228636 non-null  object        
 2   work_type            228636 non-null  object        
 3   work_subtype         228636 non-null  object        
 4   start_date           228636 non-null  datetime64[ns]
 5   end_date             217100 non-null  datetime64[ns]
 6   completed_work_area  228636 non-null  object        
 7   main_tech            228636 non-null  object        
 8   add_tech             150610 non-null  object        
 9   mech                 228636 non-null  object        
 10  proc_depth           42139 non-null   object        
 11  prod_purpose         33861 non-null   object        
 12  prod_weight          228636 non-null  object        
 13  prod_dry_perc 

In [4]:
columns_to_convert = [
    'yield_fact', 'completed_work_area', 
    'field_area', 'yield_plan', 'p2o5', 'k2o', 'humus', 'ph', 
    'inv_item_weight', 'inv_item_dose', 'proc_depth', 'prod_purpose', 
    'prod_weight', 'prod_dry_perc', 'prod_humid', 'prod_impur'
]

for column in columns_to_convert:
    if column in df.columns:
        df[column] = df[column].str.replace(',', '.').apply(pd.to_numeric, errors='coerce')

In [5]:
df['yield_fact'].value_counts()

yield_fact
0.00       192230
1.18          169
1.27          167
1.30          166
1.06          165
            ...  
12.06           1
1242.28         1
19.77           1
69.58           1
752.68          1
Name: count, Length: 2545, dtype: int64

In [9]:
df = df[df['yield_fact'] > 0].reset_index()

In [10]:
df.columns

Index(['index', 'season', 'field', 'work_type', 'work_subtype', 'start_date',
       'end_date', 'completed_work_area', 'main_tech', 'add_tech', 'mech',
       'proc_depth', 'prod_purpose', 'prod_weight', 'prod_dry_perc',
       'prod_humid', 'prod_impur', 'yield_fact', 'work_id', 'region',
       'land_type', 'fertility_score', 'crop', 'crop_predecessor',
       'crop_yearold', 'field_area', 'hybrid', 'reproduction', 'seed_origin',
       'technology', 'yield_plan', 'p2o5', 'k2o', 'humus', 'ph',
       'inv_item_type', 'inv_item_name', 'inv_item_weight', 'inv_item_unit',
       'inv_item_dose', 'inv_item_date'],
      dtype='object')

In [13]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

df['yield_fact'] = pd.to_numeric(df['yield_fact'], errors='coerce')

df = df.dropna(subset=['yield_fact'])

numerical_features = df.select_dtypes(include=[np.number]).columns.tolist()

categorical_features = df.select_dtypes(include=[object]).columns
label_encoders = {}

columns_to_exclude = ['start_date', 'end_date', 'inv_item_date']
df = df.drop(columns=columns_to_exclude, errors='ignore')

for col in categorical_features:
    le = LabelEncoder()
    df[col] = df[col].fillna('Unknown')
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

features = df.drop(columns=['yield_fact', 'index'])
target = df['yield_fact']

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

model = RandomForestRegressor(random_state=42, n_estimators=100)
model.fit(X_train, y_train)

feature_importances = pd.DataFrame({
    'Feature': features.columns,
    'Importance': model.feature_importances_
}).sort_values(by='Importance', ascending=False)

print(feature_importances)

                Feature    Importance
10          prod_weight  7.462133e-01
4   completed_work_area  1.463800e-01
3          work_subtype  1.979631e-02
14              work_id  1.946928e-02
21           field_area  1.224058e-02
26           yield_plan  1.038273e-02
18                 crop  8.023182e-03
7                  mech  5.771647e-03
1                 field  5.701408e-03
27                 p2o5  5.540703e-03
29                humus  4.635090e-03
28                  k2o  2.937715e-03
30                   ph  2.937525e-03
12           prod_humid  2.545048e-03
22               hybrid  1.354662e-03
19     crop_predecessor  1.156115e-03
5             main_tech  1.035174e-03
11        prod_dry_perc  9.136157e-04
6              add_tech  7.598409e-04
17      fertility_score  6.169858e-04
23         reproduction  5.394398e-04
24          seed_origin  4.145509e-04
13           prod_impur  3.096321e-04
0                season  1.546870e-04
15               region  9.898084e-05
25          